In [ ]:
# default_exp data_man
from nbdev import *
%reload_ext autoreload
%autoreload 2

# Data management
> Create, from FT1 and FT2, a compact data set with photon and livetime info.

### Overview

Fermi-LAT weekly data files are extracted from the [GSFC FTP server ](https://heasarc.gsfc.nasa.gov/FTP/fermi/data/lat/weekly), 
with subfolders for the photon data, `photon` and spacecraft data, `spacecraft`. It is [described here](http://fermi.gsfc.nasa.gov/ssc/data/access/http://fermi.gsfc.nasa.gov/ssc/data/access/)

The class `WeeklyData` downloads these to temporary files and constructs a dict for each week with
contents

* photons: a table, entry per selected photon with columns 
  * time since the weekly tstart (float32)
  * energy and event type (uint8)
  * position as HEALPix index (uint32)
* sc_data: a table, an entry per 30-s interval, with columns, all float32
  * start/stop time 
  * S/C direction 
  * zenith direction
* gti_times: an array of interleaved start/stop intervals

These dict objects, one per week, are saved in a folder

####  Use run number for offset instead of week time
Save run number (int32) as "sparse array" 
A run is at most 6 ks. If I integerize the offset from this with 32 bits, I have 5e5 intervals/s, 2 us. 

In [ ]:
# export
import os
from astropy.io import fits

import healpy
from pathlib import Path
import numpy as np
import pandas as pd
pd.set_option('display.precision', 2)
import pickle
from wtlike.config import *

In [ ]:
# export
def get_ft1_data( config, ft1_file):

        """
        Read in a photon data (FT1) file, bin in energy and position to convert to a compact DataFrame
        
        - `ft1_file` -- A monthly file generated by J. Ballet, or a weekly file from GSFC
        
        Depends on config items
        - `theta_cut, z_cut` -- selection criteria
        - `ebins, etypes` -- define band index
        - `nside, nest` -- define HEALPix binning
         
        Returns a tuple with 

        - `tstart`, the start MET time

        - DataFrame  with columns
           - `band` (uint8):    energy band index*2 + 0,1 for Front/Back 
           - `nest_index`  if nest else `ring_index` (uint32): HEALPIx index for the nside 
           - `run_diff`   Run number difference from previous entry. (Saved as spase arary)
           - `time` (float32):    the elapsed time in s from header value TSTART in the FT1 file
           - `rtime` (float32): MET time relative to run id.
           
        - gti times as an interleaved start, stop array.  
        
        For the selected events above 100 MeV, this represents 9 bytes per photon, vs. 27.
        """
      
        ebins = config.energy_edges
        etypes = config.etypes
        nside = config.nside
        nest = config.nest
        z_cut =config.z_max
        theta_cut = np.degrees(np.arccos(config.cos_theta_max))
        verbose = config.verbose
        
        with  fits.open(ft1_file) as ft1:
            tstart = ft1[1].header['TSTART'] 

            ## GTI - setup raveled array function to make cut
            gti_data= ft1['GTI'].data
            # extract arrays for values of interest
            data =ft1['EVENTS'].data

        a,b = sorted(gti_data.START), sorted(gti_data.STOP)
        
        gti_times = np.ravel(np.column_stack((a,b)))
        if np.any(np.diff(gti_times)<0):
            raise Exception(f'Non-monatonic GTI found')
            
        def apply_gti(time):
            x = np.digitize(time, gti_times)
            return np.bitwise_and(x,1).astype(bool)
        
        # apply  selections
        
        sel =  ((data['ENERGY'] > ebins[0]) & 
                (data['ZENITH_ANGLE'] < z_cut) & 
                (data['THETA'] < theta_cut))
    
        dsel = data[sel]

        # get the columns for output
        glon, glat, energy, et, z, theta, time, ec =\
             [dsel[x] for x in 'L B ENERGY EVENT_TYPE ZENITH_ANGLE THETA TIME EVENT_CLASS'.split()]
        
        # generate event_type masks
        et_mask={}
        for ie in etypes:
            et_mask[ie]= et[:,-1-ie]
            
       
        if verbose>1:
            total = sum(b)-sum(a)
            fraction = total/(b[-1]-a[0])
  
            print(  f'FT1: {ft1_file.name}, GTI range {a[0]:.1f}-{b[-1]:.1f}:  {len(data):,} photons'\
                    f'\n\tSelection E > {ebins[0]:.0f} MeV. theta<{theta_cut:.1f} and z<{z_cut} remove:'\
                    f' {100.- 100*len(dsel)/float(len(data)):.2f}%'
#                     f', GTI cut removes {sum(~gti_cut)}'
                 )

        
        # event class -- turn into single int for later mask
#         bits = np.array([1<<n for n in range(20)])
#         def to_bin(x):
#             return np.sum(bits[x[:20]])
#         ec = [to_bin(row[20]) for row in ec
        
        # pixelate direction
        hpindex = healpy.ang2pix(nside, glon, glat, nest=nest, lonlat=True).astype(np.uint32)
        hpname = 'nest_index' if nest else 'ring_index'
        
        # digitize energy and create band index incluing (front/back)
        band_index = (2*(np.digitize(energy, ebins, )-1) + et_mask[1]).astype(np.uint8)

        # sparcify the run, and make sparsified array of diffs 
        run_id = dsel['RUN_ID']
        rundiff = np.diff(run_id, prepend=0).astype(np.int32)
#         if np.any(rundiff<0):
#             print(f'\t*** {sum(rundiff<0)} runs out of order: will need to sort.')
        rdspar = pd.arrays.SparseArray(rundiff)
        
        d ={    'band'  : band_index,
                hpname  : hpindex,
                'time'  : (time-tstart).astype(np.float32),
                'run_diff':rdspar,
                'rtime' : (time-run_id).astype(np.float32),
                ### following for testing
                #'run_id': run_id.astype(np.int32),
                #'met'   : time.astype(float),
           }             
        if verbose>1:
            print(f'\tReturning tstart={tstart:.0f}, {len(dsel):,} photons.')
            
        return  tstart, pd.DataFrame(d), gti_times

In [ ]:
show_doc(get_ft1_data, title_level=2)

<h2 id="get_ft1_data" class="doc_header"><code>get_ft1_data</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h2>

> <code>get_ft1_data</code>(**`config`**, **`ft1_file`**)

Read in a photon data (FT1) file, bin in energy and position to convert to a compact DataFrame

- `ft1_file` -- A monthly file generated by J. Ballet, or a weekly file from GSFC

Depends on config items
- `theta_cut, z_cut` -- selection criteria
- `ebins, etypes` -- define band index
- `nside, nest` -- define HEALPix binning
 
Returns a tuple with 

- `tstart`, the start MET time

- DataFrame  with columns
   - `band` (uint8):    energy band index*2 + 0,1 for Front/Back 
   - `nest_index`  if nest else `ring_index` (uint32): HEALPIx index for the nside 
   - `run_diff`   Run number difference from previous entry. (Saved as spase arary)
   - `time` (float32):    the elapsed time in s from header value TSTART in the FT1 file
   - `rtime` (float32): MET time relative to run id.
   
- gti times as an interleaved start, stop array.  

For the selected events above 100 MeV, this represents 9 bytes per photon, vs. 27.

In [ ]:
# export
def get_ft2_info(config, filename, 
                 gti=lambda t: True):
    """Process a FT2 file, with S/C history data, and return a summary DataFrame
    
    Parameters:

    * config -- verbose, cos_theta_max, z_max        
    * filename -- spacecraft (FT2) file       
    * gti -- GTI object that checkes for allowed intervals, in MJD units

    Returns: A DataFrame with fields consistent with GTI if specified 
        
    * start, stop -- interval in MJD units
    * livetime -- sec
    * ra_scz, dec_scz --spaceraft direction
    * ra_zenith, dec_zenith -- local zenith
    """
    # combine the files into a DataFrame with following fields besides START and STOP (lower case for column)
    fields    = ['LIVETIME','RA_SCZ','DEC_SCZ', 'RA_ZENITH','DEC_ZENITH']
    with fits.open(filename) as hdu:
        scdata = hdu['SC_DATA'].data
        tstart, tstop = [float(hdu[0].header[field]) for field in  ('TSTART','TSTOP') ]
    
    if config.verbose>1:
        print(f'FT2: {filename.name}, MET range {tstart:.1f}-{tstop:.1f},', end='')# {"not" if gti is None else ""} applying GTI')
        
    # get times to check against MJD limits and GTI
    start, stop = [MJD(np.array(scdata.START, float)),
                   MJD(np.array(scdata.STOP, float))]
    
    # apply GTI to bin center (avoid edge effects?)
    in_gti = gti(0.5*(start+stop)) 
    if config.verbose>1:
        s = sum(in_gti)
        print(f' {len(start)} entries, {s} ({100*s/len(start):.1f}%) in GTI')
    
    t = [('start', start[in_gti]), ('stop',stop[in_gti])]+\
        [(field.lower(), np.array(scdata[field][in_gti],np.float32)) for field in fields ]

    sc_data = pd.DataFrame(dict(t) ) 
            
    return sc_data

In [ ]:
show_doc(get_ft2_info, title_level=2)

<h2 id="get_ft2_info" class="doc_header"><code>get_ft2_info</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h2>

> <code>get_ft2_info</code>(**`config`**, **`filename`**, **`gti`**=*`'<lambda>'`*)

Process a FT2 file, with S/C history data, and return a summary DataFrame

Parameters:

* config -- verbose, cos_theta_max, z_max        
* filename -- spacecraft (FT2) file       
* gti -- GTI object that checkes for allowed intervals, in MJD units

Returns: A DataFrame with fields consistent with GTI if specified 
    
* start, stop -- interval in MJD units
* livetime -- sec
* ra_scz, dec_scz --spaceraft direction
* ra_zenith, dec_zenith -- local zenith

In [ ]:
# # hide
# filename =Path('/tmp/from_gsfc/week668_ft2.fits')
# if os.path.exists(filename):
#     config = Config()
#     config.verbose=3
#     t = get_ft2_info(config, filename); 
#     print(t.head())
# else:
#     print(f'File {filename} does not exist, skipping get_ft2_info test')

In [ ]:
# export
class WeeklyData(object):
    """Download and process weekly Fermi-LAT files from GSFC,
    
    at FTP 'https://heasarc.gsfc.nasa.gov/FTP/fermi/data/lat/weekly' 
    
    * week: a mission week number
    * saveto: path to save the files
    
    Creates a pickled file as  dict with
    
    * tstart--start time (MJD)
    * photons -- DataFrame with photon data--see `get_ft1_data`
    * sc_data -- DataFrame with Spacecraft data--see `get_ft2_info` 
    * gti_times -- array of interleaved start/stop times from the photon data 
    """
    
    ftp = 'https://heasarc.gsfc.nasa.gov/FTP/fermi/data/lat/weekly'
    tmp = Path('/tmp/from_gsfc')
    
    def __init__(self, config, week,  overwrite=False):
        """
        * week: a mission week number, starting at 9
        * saveto: path to save the files
        
        """
        import wget
        self.config= config
        self.saveto=Path(config.wtlike_data/'data_files')
        os.makedirs(self.saveto, exist_ok=True)
        assert week>8
        self.wk = week
  
        self.ft2_file, self.ft1_file = fits_files = [self.tmp/f'week{week:03d}_{x}.fits'
                                                     for x in 'ft2 ft1'.split()]
        os.makedirs(self.tmp, exist_ok=True)
        
        urls = []
        for ftype in  ['spacecraft', 'photon']:
             urls.append(f'{self.ftp}/{ftype}/lat_{ftype}_weekly_w{week:03d}_p{"305" if ftype=="photon" else "310" }_v001.fits')
        
        for url, fname in zip(urls, fits_files):
            if not fname.exists() or overwrite:
                if config.verbose>1:
                    print(f'{url.split("/")[-1]} -> {fname}')
                try:
                    if fname.exists(): os.remove(fname)
                    wget.download(str(url), str(fname))
                except Exception as msg:
                    print(f'Failed to download {url}: {msg}')
                    raise
            else: 
                if config.verbose>1: print(f'{fname} exists')

    def process_ft1(self):
        self.tstart, self.photon_data, self.gti_times = get_ft1_data(self.config, self.ft1_file)
    
    def process_ft2(self):
        
        def apply_gti(time): # note MJD
            x = np.digitize(time, MJD(self.gti_times))
            return np.bitwise_and(x,1).astype(bool)
        
        self.sc_data = get_ft2_info(self.config, self.ft2_file, apply_gti)
        
    def save(self):
        """process, then save aa dict"""
        
        self.process_ft1()
        self.process_ft2()
        
        d = dict(tstart = self.tstart,
                photons = self.photon_data,
                sc_data = self.sc_data,
                gti_times = self.gti_times)
        filename = self.saveto/f'week_{self.wk:03d}.pkl'
        pickle.dump(d, open(filename, 'wb'))
        if self.config.verbose>0:
            print(f'Saved to {filename}')
        

In [ ]:
# WeeklyData(Config(), 674).save()

In [ ]:
show_doc(WeeklyData)

<h2 id="WeeklyData" class="doc_header"><code>class</code> <code>WeeklyData</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>WeeklyData</code>(**`config`**, **`week`**, **`overwrite`**=*`False`*)

Download and process weekly Fermi-LAT files from GSFC,

at FTP 'https://heasarc.gsfc.nasa.gov/FTP/fermi/data/lat/weekly' 

* week: a mission week number
* saveto: path to save the files

Creates a pickled file as  dict with

* tstart--start time (MJD)
* photons -- DataFrame with photon data--see [`get_ft1_data`](/wtlikedata_man#get_ft1_data)
* sc_data -- DataFrame with Spacecraft data--see [`get_ft2_info`](/wtlikedata_man#get_ft2_info) 
* gti_times -- array of interleaved start/stop times from the photon data 

## check the weekly files

In [ ]:
# export
def data_check(config=None):
    """
    Return: list of files, last week number, number of days in last week"""
    if config is None: config=Config()
    if config.valid:    
        weekly_folder = config.wtlike_data/'data_files'
        ff = sorted(list(weekly_folder.glob('*.pkl')))
        if len(ff)==0:
            print(f'No .pkl files found in {weekly_folder}', file=sys.stderr)
            return
        wk = list(map(lambda f: int(os.path.splitext(f)[0][-3:]), ff))
        lastweek = pickle.load(open(ff[-1],'rb'))
        gti = lastweek['gti_times'];
        days = (gti[-1]-gti[0])/(24*3600)
        if config.verbose>0:
            print(f'Weekly folder "{weekly_folder}" contains {len(wk)} weeks.'\
            f'\n\t Last week, # {wk[-1]}, has {days:.1f} days, ends at UTC {UTC(MJD(gti[-1]))}' )
        return ff, wk[-1], days
    else:
        print(f'Config not valid, {config.errors} no files found', file=sys.stderr)
        return None

def check_data(config=None):
    data_check(config)

In [ ]:
# export
def update_recent(config=None, test=False):
    if config is None: config=Config()
    ff, last_wk, days = data_check(config);
    if days<6:
        wk = last_wk
        print(f'Will reload week {last_wk} ...')
    else:
        wk = last_wk+1
        print(f'Will load next week, # {last_wk+1} ...')
    if not test:
        WeeklyData(config, wk, overwrite=True).save()
        _,last_wk, new_days = data_check(config)
        #print(f'Now {days}')
        return new_days!=days
    else: 
        print('... but testing')
        return False

In [ ]:
# #hide
#data_check()
#update_recent();

In [ ]:
#export    
def get_data_files(config=None):
    """
    Return a list of the pickled data files
    """
    if config.valid:    
        weekly_folder = config.wtlike_data/'data_files'
        ff = sorted(list(weekly_folder.glob('*.pkl')))
        if len(ff)==0:
            print(f'No .pkl files found in {weekly_folder}', file=sys.stderr)
            return
        wk = list(map(lambda f: int(os.path.splitext(f)[0][-3:]), ff))
        lastweek = pickle.load(open(ff[-1],'rb'))
        if config.verbose>0:
            gti = lastweek['gti_times'];
            days = (gti[-1]-gti[0])/(24*3600)
            print(f'Weekly folder "{weekly_folder}" contains {len(wk)} weeks,  {wk[0]} to {wk[-1]},'\
            f' last week has {days:.1f} days, ends at {UTC(MJD(gti[-1]))}' )
        return ff
    else:
        print(f'Config not valid, {config.errors} no files found', file=sys.stderr)
        return []
    
    
def get_week(wk, config=None):
    """download GSFC data for the week #"""
    if config is None: config =Config()
    assert config.valid
    WeeklyData(config, wk).save()

In [ ]:
# WeeklyData(config,676, overwrite=True).save()

In [ ]:
# config = Config()
# ff=get_data_files(config); ff[-1]

In [ ]:
# export
# class GTI(object):
    
#     def __init__(self, gti_times):
#         self.mjd_gti = MJD(gti_times)
    
#     def __call__(self, time:'float'):
#         x = np.digitize(time, self.mjd_gti)
#         return np.bitwise_and(x,1).astype(bool)

In [ ]:
# hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_data_man.ipynb.
Converted 02_effective_area.ipynb.
Converted 03_weights.ipynb.
Converted 04_exposure.ipynb.
Converted 04_simulation.ipynb.
Converted 05_source_data.ipynb.
Converted 06_poisson.ipynb.
Converted 07_loglike.ipynb.
Converted 08_cell_data.ipynb.
Converted 09_lightcurve.ipynb.
Converted 14_bayesian.ipynb.
Converted 90_main.ipynb.
Converted 99_tutorial.ipynb.
Converted index.ipynb.
Mon May 31 05:52:00 PDT 2021
